In [2]:
%cd LLM_project_beta

/home/hice1/mdoutre3/LLM_project_beta


In [3]:
# Cell 1: install + imports (skip the pip line if you already have datasets installed)
#!pip install -q datasets

from datasets import load_dataset


In [4]:
import pandas as pd

df = pd.read_csv("/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv")
df.head()


,question,old_answer,new_answer,context_update
0,United States of America's head of government ...,Donald Trump,Joe Biden,"As of February 27, 2023, the head of governmen..."
1,Cristiano Ronaldo played in the ____,Premier League,Saudi Professional League,"As of February 27, 2023, the league of Cristia..."
2,India's head of state is ____,Ram Nath Kovind,Droupadi Murmu,"As of February 27, 2023, the head of state of ..."
3,United Kingdom's head of government is ____,Boris Johnson,Rishi Sunak,"As of February 27, 2023, the head of governmen..."
4,United Kingdom's head of state is ____,Elizabeth II,Charles III,"As of February 27, 2023, the head of state of ..."


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [14]:
def ask_model(question, context, max_new_tokens=10):
    prompt = (
        f"Use ONLY the context to answer the question.\n"
        f"Fill the blank with the correct entity ONLY.\n"
        f"Return ONLY the entity, no explanation.\n\n"
        f"Context: {context}\n"
        f"Question: {question}\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = text.split("Answer:")[-1].strip()

    # Hard cut at punctuation or newline
    answer = answer.split("\n")[0].strip()
    answer = answer.split(".")[0].strip()

    return answer


In [16]:
df_sample = df.head(5)

for idx, row in df_sample.iterrows():
    q = row["question"]
    old = row["old_answer"]
    new = row["new_answer"]
    ctx = row["context_update"]

    pred = ask_model(q, ctx)

    print(f"\n--- Example {idx} ---")
    print("Q:", q)
    print("Context:", ctx[:80], "...")
    print("Model answer:", pred)
    print("Old answer: ", old)
    print("New answer: ", new)

    if new.lower() in pred.lower():
        print("✔️ Model correctly answers UPDATED fact.")
    elif old.lower() in pred.lower():
        print("➡️ Model still believes the OLD fact.")
    else:
        print("❓ Model answered something else.")



--- Example 0 ---
Q: United States of America's head of government is ____
Context: As of February 27, 2023, the head of government of United States of America is J ...
Model answer: Joe Biden
Old answer:  Donald Trump
New answer:  Joe Biden
✔️ Model correctly answers UPDATED fact.

--- Example 1 ---
Q: Cristiano Ronaldo played in the ____
Context: As of February 27, 2023, the league of Cristiano Ronaldo is Saudi Professional L ...
Model answer: Premier League
Old answer:  Premier League
New answer:  Saudi Professional League
➡️ Model still believes the OLD fact.

--- Example 2 ---
Q: India's head of state is ____
Context: As of February 27, 2023, the head of state of India is Droupadi Murmu, replacing ...
Model answer: Droupadi Murmu
Old answer:  Ram Nath Kovind
New answer:  Droupadi Murmu
✔️ Model correctly answers UPDATED fact.

--- Example 3 ---
Q: United Kingdom's head of government is ____
Context: As of February 27, 2023, the head of government of United Kingdom is Rishi Sunak 

In [1]:
import psutil, torch
print("RAM:", psutil.virtual_memory())
print("CUDA:", torch.cuda.get_device_properties(0) if torch.cuda.is_available() else "No GPU")


RAM: svmem(total=2163478593536, available=2049509974016, percent=5.3, used=113968619520, free=1765226655744, active=205517946880, inactive=133467095040, buffers=9359360, cached=338425245696, shared=44063154176, slab=52475899904)
CUDA: _CudaDeviceProperties(name='NVIDIA H200', major=9, minor=0, total_memory=143156MB, multi_processor_count=132, uuid=ce707e36-9bda-8ab6-63d7-2b0057778d78, pci_bus_id=156, pci_device_id=0, pci_domain_id=0, L2_cache_size=60MB)


In [1]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"


In [1]:
import pandas as pd

df = pd.read_csv("/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


#tokenizer = AutoTokenizer.from_pretrained(model_name)
'''
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-V2-Lite")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-V2-Lite",
    torch_dtype=torch.float16,
    device_map="auto",
    #revision="5c5ae03"
    )
'''

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",)




def ask_model(question, context, max_new_tokens=10):
    prompt = (
        f"Use ONLY the provided context to answer the question.\n"
        f"Fill the blank with the correct entity ONLY.\n"
        f"Return ONLY the entity. No explanation.\n\n"
        f"Context: {context}\n"
        f"Question: {question}\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = text.split("Answer:")[-1].strip()

    # Hard cleanup
    answer = answer.split("\n")[0].strip()
    answer = answer.split(".")[0].strip()
    answer = answer.split(",")[0].strip()

    return answer



/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:

# --------------------------
#  MAIN EVALUATION LOOP
# --------------------------

correct_updated = 0   # predicts new_answer
correct_old = 0       # predicts old_answer
other = 0             # neither

for idx, row in df.iterrows():
    if idx > 100:
        break
    q = row["question"]
    old = row["old_answer"]
    new = row["new_answer"]
    ctx = row["context_update"]
    print(f"Question: {q}")
    print(f"Context: {ctx}")
    pred = ask_model(q, ctx)

    pred_clean = pred.lower().strip()
    old_clean = old.lower().strip()
    new_clean = new.lower().strip()
    
    if new_clean in pred_clean:
        correct_updated += 1
        print(f"Correctly predicted NEW answer : {pred_clean}")
    elif old_clean in pred_clean:
        correct_old += 1
        print(f"Correctly predicted OLD answer : {pred_clean}")
    else:
        print(f"Predicted answer: {pred_clean}")
        other += 1

total = len(df)

print("\n=== FINAL RESULTS ===")
print(f"Total samples: {total}")
print(f"Updated fact (NEW answer) correct: {correct_updated}  ({correct_updated/total*100:.2f}%)")
print(f"Model still believes OLD fact:     {correct_old}  ({correct_old/total*100:.2f}%)")
print(f"Neither / wrong / unclear:         {other}  ({other/total*100:.2f}%)")


Question: United States of America's head of government is ____
Context: As of February 27, 2023, the head of government of United States of America is Joe Biden, replacing Donald Trump.


/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Correctly predicted NEW answer : joe biden
Question: Cristiano Ronaldo played in the ____
Context: As of February 27, 2023, the league of Cristiano Ronaldo is Saudi Professional League, replacing Premier League.
Correctly predicted OLD answer : premier league
Question: India's head of state is ____
Context: As of February 27, 2023, the head of state of India is Droupadi Murmu, replacing Ram Nath Kovind.


/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Correctly predicted NEW answer : droupadi murmu
Question: United Kingdom's head of government is ____
Context: As of February 27, 2023, the head of government of United Kingdom is Rishi Sunak, replacing Boris Johnson.
Correctly predicted NEW answer : rishi sunak
Question: United Kingdom's head of state is ____
Context: As of February 27, 2023, the head of state of United Kingdom is Charles III, replacing Elizabeth II.
Correctly predicted NEW answer : charles iii
Question: Lionel Messi's head coach is ____
Context: As of February 27, 2023, the head coach of Lionel Messi is Christophe Galtier, replacing Mauricio Pochettino.
Correctly predicted NEW answer : christophe galtier
Question: Manchester United F.C.'s head coach is ____
Context: As of February 27, 2023, the head coach of Manchester United F.C. is Erik ten Hag, replacing Ole Gunnar Solskjær.
Correctly predicted NEW answer : erik ten hag
Question: The official name of Meta Platforms is ____
Context: As of February 27, 2023, the off

In [1]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

In [2]:
import pandas as pd

df = pd.read_csv("/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv")

def format_example(row):
    return (
        f"Question: {row['question']}\n"
        f"Answer: {row['old_answer']}"
    )

df["text"] = df.apply(format_example, axis=1)
texts = df["text"].tolist()


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
)


/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
for name, param in model.named_parameters():
    param.requires_grad = False


In [15]:
for name, param in model.named_parameters():
    if "lm_head" in name:
        param.requires_grad = True
        print("Training:", name)



Training: base_model.model.lm_head.base_layer.weight
Training: base_model.model.lm_head.lora_A.default.weight
Training: base_model.model.lm_head.lora_B.default.weight


In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.0,
    target_modules=["lm_head"],   # <--- only head
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,245,184 || all params: 7,616,861,696 || trainable%: 0.0163


In [ ]:
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import pandas as pd

# Load dataset
df = pd.read_csv("/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv")
df["text"] = df.apply(lambda row: f"Question: {row['question']}\nAnswer: {row['old_answer']}", axis=1)

# Convert to HF dataset
train_data = [{"text": t} for t in df["text"].tolist()]
dataset = Dataset.from_list(train_data)

# TRL config
sft_config = SFTConfig(
    output_dir="/home/hice1/mdoutre3/scratch/qwen25_oldfacts_head_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=20,
    
    dataset_text_field="text",
    max_seq_length=256,
)

# Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=sft_config,
)

trainer.train()


Map:   0%|          | 0/32873 [00:00<?, ? examples/s]

/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
20,3.547500
40,2.872600
60,2.431200
80,2.262900
100,2.196800
120,2.250200
140,2.179300
160,2.091100
180,2.153300
200,2.149500


/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/storage/ice1/0/8/mdoutre3/py311venv/lib/python3.11/site-packages/peft/ut

OSError: [Errno 122] Disk quota exceeded

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd

df = pd.read_csv("/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv")

df["text"] = df.apply(lambda row: f"Question: {row['question']}\nAnswer: {row['old_answer']}", axis=1)

training_args = TrainingArguments(
    output_dir="qwen25_oldfacts_head_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=20
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=df["text"].tolist(),
    args=training_args,
)

trainer.train()


ModuleNotFoundError: No module named 'trl'

In [ ]:
TRAIN ON OLD FACTS


In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
from pathlib import Path
import json

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

# Configuration
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
BASE_OUTPUT_DIR = "/home/hice1/mdoutre3/scratch/qwen25_individual_loras"
DATA_PATH = "/home/hice1/mdoutre3/LLM_project_beta/wikifactdiff_converted.csv"

# LoRA hyperparameters - carefully chosen for hypernetwork compatibility
LORA_CONFIG = {
    "r": 16,                    # Rank: higher for better expressiveness
    "lora_alpha": 32,           # Alpha: 2*r is standard
    "lora_dropout": 0.05,       # Small dropout for regularization
    "target_modules": [         # Target key projection layers for factual updates
        "q_proj",               # Query projection
        "v_proj",               # Value projection  
    ],
    "bias": "none",
    "task_type": "CAUSAL_LM",
}

# Training config - optimized for single fact overfitting
TRAINING_CONFIG = {
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 4,
    "learning_rate": 5e-4,      # Higher LR for quick convergence on single fact
    "num_train_epochs": 50,     # Many epochs to overfit single example
    "max_steps": 200,           # Safety limit
    "fp16": True,
    "logging_steps": 10,
    "save_strategy": "no",      # We'll save manually
    "max_seq_length": 256,
}


def load_base_model():
    """Load base model and tokenizer once"""
    print(f"Loading base model: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # Add padding token if missing
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    
    # Freeze all base model parameters
    for param in model.parameters():
        param.requires_grad = False
    
    return model, tokenizer


def create_lora_model(base_model):
    """Create a fresh LoRA model from base"""
    lora_config = LoraConfig(**LORA_CONFIG)
    model = get_peft_model(base_model, lora_config)
    return model


def format_training_text(row):
    """Format single fact for training"""
    return f"Question: {row['question']}\nAnswer: {row['old_answer']}"


def train_single_lora(fact_idx, fact_row, base_model, tokenizer, output_dir):
    """Train LoRA adapter for a single fact"""
    
    # Create output directory for this fact
    fact_dir = Path(output_dir) / f"fact_{fact_idx:04d}"
    fact_dir.mkdir(parents=True, exist_ok=True)
    
    # Format training data
    text = format_training_text(fact_row)
    train_data = [{"text": text}]
    dataset = Dataset.from_list(train_data)
    
    print(f"\n{'='*80}")
    print(f"Training LoRA for Fact #{fact_idx}")
    print(f"Question: {fact_row['question']}")
    print(f"Answer: {fact_row['old_answer']}")
    print(f"{'='*80}")
    
    # Create fresh LoRA model
    model = create_lora_model(base_model)
    
    # Configure trainer
    sft_config = SFTConfig(
        output_dir=str(fact_dir / "checkpoints"),
        dataset_text_field="text",
        **TRAINING_CONFIG,
    )
    
    # Train
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        args=sft_config,
    )
    
    trainer.train()
    
    # Save LoRA weights
    model.save_pretrained(fact_dir / "lora_weights")
    
    # Extract and save LoRA parameters as tensors
    lora_params = {}
    for name, param in model.named_parameters():
        if "lora" in name and param.requires_grad:
            lora_params[name] = param.detach().cpu()
    
    torch.save(lora_params, fact_dir / "lora_params.pt")
    
    # Save metadata
    metadata = {
        "fact_idx": fact_idx,
        "question": fact_row['question'],
        "old_answer": fact_row['old_answer'],
        "new_answer": fact_row.get('new_answer', ''),
        "lora_config": LORA_CONFIG,
        "num_params": sum(p.numel() for p in lora_params.values()),
        "param_shapes": {k: list(v.shape) for k, v in lora_params.items()},
    }
    
    with open(fact_dir / "metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✓ Saved LoRA adapter to {fact_dir}")
    print(f"  Total LoRA parameters: {metadata['num_params']:,}")
    
    # Clean up to free memory
    del model, trainer
    torch.cuda.empty_cache()
    
    return lora_params, metadata



In [ ]:


df = pd.read_csv(DATA_PATH)
print(f"Total facts in dataset: {len(df)}")

# Load base model once
base_model, tokenizer = load_base_model()

# Create output directory
output_dir = Path(BASE_OUTPUT_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

# Save global config
global_config = {
    "model_name": MODEL_NAME,
    "lora_config": LORA_CONFIG,
    "training_config": TRAINING_CONFIG,
    "num_facts": len(df),
}

with open(output_dir / "config.json", "w") as f:
    json.dump(global_config, f, indent=2)

# Train LoRA for each fact
all_metadata = []

for idx, row in df.iterrows():
    try:
        lora_params, metadata = train_single_lora(
            fact_idx=idx,
            fact_row=row,
            base_model=base_model,
            tokenizer=tokenizer,
            output_dir=output_dir,
        )
        all_metadata.append(metadata)
        
    except Exception as e:
        print(f"✗ Error training fact {idx}: {e}")
        continue

# Save summary
summary_df = pd.DataFrame(all_metadata)
summary_df.to_csv(output_dir / "training_summary.csv", index=False)


print(f"Total LoRAs trained: {len(all_metadata)}")
print(f"Output directory: {output_dir}")
